In [1]:
import jax
import jax.numpy as jnp
from typing import Any

In [2]:
import treescope
from penzai import pz
from penzai.models import simple_mlp
from penzai.models.transformer.variants import llamalike_common as llamalike_transformer

In [3]:
treescope.basic_interactive_setup(autovisualize_arrays=True)

In [4]:
mlp = simple_mlp.MLP.from_config(
    name="mlp",
    init_base_rng=jax.random.key(0),
    feature_sizes=[8, 32, 32, 8]
)

mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=Parameter(label='mlp/Affine_0/Linear.weights', value=<NamedArray float32(| features:8, features_out:32) ≈-0.017 ±0.23 [≥-0.39, ≤0.38] nonzero:256 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=Parameter(label='mlp/Affine_0/AddBias.bias', value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, metadata={}), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(label='mlp/Affine_1/Linear.weights', value=<NamedArray float32(| features:32, features_out:32) ≈-0.012 ±0.17 [≥-0.31, ≤0.3] nonzero:1_024 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(label='mlp/Affine_1/AddBias.bias', value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, metadata={}), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(label='mlp/Affine_2/Linear.weights', value=<NamedArray float32(| features:32, features_out:8) ≈-0.008 ±0.22 [≥-0.39, ≤0.39] nonzero:256 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(label='mlp/Affine_2/AddBias.bias', value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, metadata={}), new_axis_names=())],
    ),
  ],
)

In [5]:
@pz.pytree_dataclass
class SaveActivations(pz.nn.Layer):
  saved_activations: pz.StateVariable[list[Any]]
  def __call__(self, activations: Any, **unused_side_inputs) -> Any:
    self.saved_activations.value = self.saved_activations.value + [activations]
    return activations

In [6]:
mlp_activations = pz.StateVariable(value=[], label="activations")

saving_model = (
    pz.select(mlp)
    .at_instances_of(pz.nn.Elementwise)
    .insert_after(SaveActivations(mlp_activations))
)

saving_model

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(label='mlp/Affine_0/Linear.weights', value=<NamedArray float32(| features:8, features_out:32) ≈-0.017 ±0.23 [≥-0.39, ≤0.38] nonzero:256 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(label='mlp/Affine_0/AddBias.bias', value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, metadata={}), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    SaveActivations(
      saved_activations=StateVariable(label='activations', value=[], metadata={}), # Repeated python obj at 0x7ff62863d6a0
    ),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(label='mlp/Affine_1/Linear.weights', value=<NamedArray float32(| features:32, features_out:32) ≈-0.012 ±0.17 [≥-0.31, ≤0.3] nonzero:1_024 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(label='mlp/Affine_1/AddBias.bias', value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, metadata={}), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    SaveActivations(
      saved_activations=StateVariable(label='activations', value=[], metadata={}), # Repeated python obj at 0x7ff62863d6a0
    ),
    Affine(sublayers=[Linear(weights=Parameter(label='mlp/Affine_2/Linear.weights', value=<NamedArray float32(| features:32, features_out:8) ≈-0.008 ±0.22 [≥-0.39, ≤0.39] nonzero:256 (wrapping jax.Array)>, metadata={}), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(label='mlp/Affine_2/AddBias.bias', value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, metadata={}), new_axis_names=())]),
  ],
)

In [7]:
input = pz.nx.ones({"features": 8})
output = saving_model(input)
mlp_activations

StateVariable(label='activations', value=[<NamedArray float32(| features:32) ≈0.27 ±0.44 [≥0.0, ≤1.8] zero:20 nonzero:12 (wrapping jax.Array)>, <NamedArray float32(| features:32) ≈0.1 ±0.21 [≥0.0, ≤0.74] zero:22 nonzero:10 (wrapping jax.Array)>], metadata={})

In [8]:
vocab_size = 8

config = llamalike_transformer.LlamalikeTransformerConfig(
    num_kv_heads=2,
    query_head_multiplier=2,
    embedding_dim=16,
    projection_dim=16,
    mlp_hidden_dim=16,
    num_decoder_blocks=2,
    vocab_size=vocab_size,
    mlp_variant='geglu_approx',
    tie_embedder_and_logits=True,
)
transformer = llamalike_transformer.build_llamalike_transformer(config, name="llama_like")
transformer

TransformerLM(
  body=Sequential(
    sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=ParameterSlot(label='llama_like/embedder.embeddings'), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(4., dtype=float32)>), TransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_0/pre_attention_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), Attention(input_to_query=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/query.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'query_heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions_input_name='token_positions'), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/key.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions_input_name='token_positions')]), input_to_value=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/value.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'projection': 'p'}, {'seq': 'tkv', 'head_groups': 'hg', 'projection': 'p'}), output_axes={'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'kv_seq': 'tkv'}), ApplyCausalAttentionMask(masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>, query_positions_input_name='token_positions', kv_positions_input_name='token_positions', seq_axis='seq', kv_seq_axis='kv_seq'), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'head_groups': 'hg', 'projection': 'p'}), output_axes={'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'projection': 'p'}), Linear(weights=ParameterSlot(label='llama_like/block_0/attention/output.weights'), in_axis_names=('head_groups', 'query_heads', 'projection'), out_axis_names=('embedding',))]))])), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_0/pre_ffw_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), TransformerFeedForward(sublayers=[BranchAndMultiplyTogether(branches=[NamedGroup(name='gate', sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/gating_linear.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',)), Elementwise(fn=functools.partial(<function gelu at 0x7ff68227f740>, approximate=True))]), Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/value_linear.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',))]), Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/out_linear.weights'), in_axis_names=('neurons',), out_axis_names=('embedding',))])]))]), TransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_1/pre_attention_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), Attention(input_to_query=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_1/attention/query.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'query_heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions_input_name='token_positions'), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=

In [9]:
transformer_activations = pz.StateVariable(value=[], label="transformer_activations")

saving_transformer = (
    pz.select(transformer)
    .at_instances_of(pz.nn.Residual)
    .insert_after(SaveActivations(transformer_activations))
)

saving_transformer

TransformerLM(
  body=Sequential(
    sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=ParameterSlot(label='llama_like/embedder.embeddings'), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(4., dtype=float32)>), TransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_0/pre_attention_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), Attention(input_to_query=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/query.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'query_heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions_input_name='token_positions'), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/key.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions_input_name='token_positions')]), input_to_value=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/attention/value.weights'), in_axis_names=('embedding',), out_axis_names=('head_groups', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'projection': 'p'}, {'seq': 'tkv', 'head_groups': 'hg', 'projection': 'p'}), output_axes={'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'kv_seq': 'tkv'}), ApplyCausalAttentionMask(masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>, query_positions_input_name='token_positions', kv_positions_input_name='token_positions', seq_axis='seq', kv_seq_axis='kv_seq'), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'head_groups': 'hg', 'projection': 'p'}), output_axes={'seq': 'tq', 'head_groups': 'hg', 'query_heads': 'hq', 'projection': 'p'}), Linear(weights=ParameterSlot(label='llama_like/block_0/attention/output.weights'), in_axis_names=('head_groups', 'query_heads', 'projection'), out_axis_names=('embedding',))]))])), SaveActivations(saved_activations=StateVariable(label='transformer_activations', value=[], metadata={})), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_0/pre_ffw_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), TransformerFeedForward(sublayers=[BranchAndMultiplyTogether(branches=[NamedGroup(name='gate', sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/gating_linear.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',)), Elementwise(fn=functools.partial(<function gelu at 0x7ff68227f740>, approximate=True))]), Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/value_linear.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',))]), Linear(weights=ParameterSlot(label='llama_like/block_0/mlp/out_linear.weights'), in_axis_names=('neurons',), out_axis_names=('embedding',))])])), SaveActivations(saved_activations=StateVariable(label='transformer_activations', value=[], metadata={}))]), TransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=ParameterSlot(label='llama_like/block_1/pre_attention_norm/scale.weights'), in_axis_names=(), out_axis_names=())]), Attention(input_to_query=Sequential(sublayers=[Linear(weights=ParameterSlot(label='llama_like/block_1/attention/query.weights'), in_axis_names=('embedding',), out_axis_names=('head

In [10]:

key = jax.random.PRNGKey(0)
batch_size = 4
sequence_length = 16
sequences = jax.random.randint(key, (batch_size, sequence_length), 0, vocab_size)
input = jax.nn.one_hot(sequences, num_classes=vocab_size)
input = pz.nx.wrap(input, "batch", "seq", "features")
input

<NamedArray float32(| batch:4, seq:16, features:8) ≈0.12 ±0.33 [≥0.0, ≤1.0] zero:448 nonzero:64 (wrapping jax.Array)
  <Arrayviz rendering>
| Device: GPU 0>

In [11]:
output = saving_transformer(input)
transformer_activations

UnboundVariableError: Tried to access the .value property of a ParameterSlot: ParameterSlot(label='llama_like/embedder.embeddings').